# Capstone Project
## Neural translation model
### Instructions

In this notebook, you will create a neural network that translates from English to German. You will use concepts from throughout this course, including building more flexible model architectures, freezing layers, data processing pipeline and sequence modelling.

This project is peer-assessed. Within this notebook you will find instructions in each section for how to complete the project. Pay close attention to the instructions as the peer review will be carried out according to a grading rubric that checks key parts of the project instructions. Feel free to add extra cells into the notebook as required.

### How to submit

When you have completed the Capstone project notebook, you will submit a pdf of the notebook for peer review. First ensure that the notebook has been fully executed from beginning to end, and all of the cell outputs are visible. This is important, as the grading rubric depends on the reviewer being able to view the outputs of your notebook. Save the notebook as a pdf (File -> Download as -> PDF via LaTeX). You should then submit this pdf for review.

### Let's get started!

We'll start by running some imports, and loading the dataset. For this project you are free to make further imports throughout the notebook as you wish. 

In [7]:
import tensorflow as tf
import tensorflow_hub as hub
import unicodedata
import re

![Flags overview image](data/germany_uk_flags.png)

For the capstone project, you will use a language dataset from http://www.manythings.org/anki/ to build a neural translation model. This dataset consists of over 200,000 pairs of sentences in English and German. In order to make the training quicker, we will restrict to our dataset to 20,000 pairs. Feel free to change this if you wish - the size of the dataset used is not part of the grading rubric.

Your goal is to develop a neural translation model from English to German, making use of a pre-trained English word embedding module.

In [8]:
# Run this cell to load the dataset

NUM_EXAMPLES = 20000
data_examples = []
with open('data/deu.txt', 'r', encoding='utf8') as f:
    for line in f.readlines():
        if len(data_examples) < NUM_EXAMPLES:
            data_examples.append(line)
        else:
            break

In [9]:
# These functions preprocess English and German sentences

def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r"ü", 'ue', sentence)
    sentence = re.sub(r"ä", 'ae', sentence)
    sentence = re.sub(r"ö", 'oe', sentence)
    sentence = re.sub(r'ß', 'ss', sentence)
    
    sentence = unicode_to_ascii(sentence)
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r"[^a-z?.!,']+", " ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence.strip()

#### The custom translation model
The following is a schematic of the custom translation model architecture you will develop in this project.

![Model Schematic](data/neural_translation_model.png)

Key:
![Model key](data/neural_translation_model_key.png)

The custom model consists of an encoder RNN and a decoder RNN. The encoder takes words of an English sentence as input, and uses a pre-trained word embedding to embed the words into a 128-dimensional space. To indicate the end of the input sentence, a special end token (in the same 128-dimensional space) is passed in as an input. This token is a TensorFlow Variable that is learned in the training phase (unlike the pre-trained word embedding, which is frozen).

The decoder RNN takes the internal state of the encoder network as its initial state. A start token is passed in as the first input, which is embedded using a learned German word embedding. The decoder RNN then makes a prediction for the next German word, which during inference is then passed in as the following input, and this process is repeated until the special `<end>` token is emitted from the decoder.

## 1. Text preprocessing
* Create separate lists of English and German sentences, and preprocess them using the `preprocess_sentence` function provided for you above.
* Add a special `"<start>"` and `"<end>"` token to the beginning and end of every German sentence.
* Use the Tokenizer class from the `tf.keras.preprocessing.text` module to tokenize the German sentences, ensuring that no character filters are applied. _Hint: use the Tokenizer's "filter" keyword argument._
* Print out at least 5 randomly chosen examples of (preprocessed) English and German sentence pairs. For the German sentence, print out the text (with start and end tokens) as well as the tokenized sequence.
* Pad the end of the tokenized German sequences with zeros, and batch the complete set of sequences into one numpy array.

In [10]:
english_sentences = []
german_sentences = []

for example in data_examples:
    e, g = example.split('\t')[:2]
    english_sentences.append(e)
    german_sentences.append(g)

english_sentences = list(map(preprocess_sentence, english_sentences))
german_sentences = list(map(preprocess_sentence, german_sentences))
german_sentences = list(map(lambda x: '<start> ' + x + ' <end>', german_sentences))

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(filters='')
tokenizer.fit_on_texts(german_sentences) 
tokenized_german_sentences = tokenizer.texts_to_sequences(german_sentences) 

In [12]:
import numpy as np
np.random.seed(11)
n = 5
for i in np.random.choice(len(data_examples), n):
    print(f'English: {english_sentences[i]}')
    print(f'Gegman: {german_sentences[i]}')
    print(f'Tokenized german: {tokenized_german_sentences[i]}')
    print()

English: he loves soccer .
Gegman: <start> er ist ein grosser fussballfreund . <end>
Tokenized german: [1, 14, 6, 19, 571, 4200, 3, 2]

English: school's out .
Gegman: <start> die schule ist vorbei . <end>
Tokenized german: [1, 26, 304, 6, 300, 3, 2]

English: i like cookies .
Gegman: <start> ich esse gerne kekse . <end>
Tokenized german: [1, 4, 261, 149, 1656, 3, 2]

English: tom is early .
Gegman: <start> tom ist frueh dran . <end>
Tokenized german: [1, 5, 6, 391, 338, 3, 2]

English: where's tom from ?
Gegman: <start> woher kommt tom ? <end>
Tokenized german: [1, 971, 120, 5, 7, 2]



In [13]:
max([len(tokenized_german_sentences[i]) for i in range(20000)])

14

In [14]:
len(tokenizer.index_word)

5743

In [ ]:
tokenizer.word_index['<start>']

In [15]:
padded_tokenized_german_sentences = tf.keras.preprocessing.sequence.pad_sequences(tokenized_german_sentences, 
                                                                                  maxlen=14, padding='post', 
                                                                                  value=0) 

In [16]:
padded_tokenized_german_sentences.shape

(20000, 14)

## 2. Prepare the data with tf.data.Dataset objects

#### Load the embedding layer
As part of the dataset preproceessing for this project, you will use a pre-trained English word embedding module from TensorFlow Hub. The URL for the module is https://tfhub.dev/google/tf2-preview/nnlm-en-dim128-with-normalization/1. This module has also been made available as a complete saved model in the folder `'./models/tf2-preview_nnlm-en-dim128_1'`. 

This embedding takes a batch of text tokens in a 1-D tensor of strings as input. It then embeds the separate tokens into a 128-dimensional space. 

The code to load and test the embedding layer is provided for you below.

**NB:** this model can also be used as a sentence embedding module. The module will process each token by removing punctuation and splitting on spaces. It then averages the word embeddings over a sentence to give a single embedding vector. However, we will use it only as a word embedding module, and will pass each word in the input sentence as a separate token.

In [17]:
# Load embedding module from Tensorflow Hub

#embedding_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1", 
#                                 output_shape=[128], input_shape=[], dtype=tf.string)
embedding_layer = tf.keras.models.load_model('./models/tf2-preview_nnlm-en-dim128_1')

In [18]:
# Test the layer

embedding_layer(tf.constant(["these", "aren't", "the", "droids", "you're", "looking", "for"])).shape

TensorShape([7, 128])

You should now prepare the training and validation Datasets.

* Create a random training and validation set split of the data, reserving e.g. 20% of the data for validation (NB: each English dataset example is a single sentence string, and each German dataset example is a sequence of padded integer tokens).
* Load the training and validation sets into a tf.data.Dataset object, passing in a tuple of English and German data for both training and validation sets.
* Create a function to map over the datasets that splits each English sentence at spaces. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.strings.split function._
* Create a function to map over the datasets that embeds each sequence of English words using the loaded embedding layer/model. Apply this function to both Dataset objects using the map method.
* Create a function to filter out dataset examples where the English sentence is more than 13 (embedded) tokens in length. Apply this function to both Dataset objects using the filter method.
* Create a function to map over the datasets that pads each English sequence of embeddings with some distinct padding value before the sequence, so that each sequence is length 13. Apply this function to both Dataset objects using the map method. _Hint: look at the tf.pad function. You can extract a Tensor shape using tf.shape; you might also find the tf.math.maximum function useful._
* Batch both training and validation Datasets with a batch size of 16.
* Print the `element_spec` property for the training and validation Datasets. 
* Using the Dataset `.take(1)` method, print the shape of the English data example from the training Dataset.
* Using the Dataset `.take(1)` method, print the German data example Tensor from the validation Dataset.

In [19]:
english_sentences = np.array(english_sentences)
valid_indices = np.random.choice(len(data_examples), int(0.2 * len(data_examples)))
train_indices = np.array(list(set(range(len(data_examples))) - set(valid_indices)))

input_train = english_sentences[train_indices]
target_train = padded_tokenized_german_sentences[train_indices]

input_valid = english_sentences[valid_indices]
target_valid = padded_tokenized_german_sentences[valid_indices]

In [30]:
def make_Dataset(input_array, target_array):
    return tf.data.Dataset.from_tensor_slices((input_array, target_array)) 

train_data = make_Dataset(input_train, target_train)
valid_data = make_Dataset(input_valid, target_valid)

In [31]:
def str_split(e, g):
    e = tf.strings.split(e)
    return e, g

train_data = train_data.map(str_split)
valid_data = valid_data.map(str_split)
#train_data.map(lambda x, y: (tf.strings.split(x), y))

In [32]:
def embed_english(x, y):
    return embedding_layer(x), y

train_data = train_data.map(embed_english)
valid_data = valid_data.map(embed_english)

In [33]:
def remove_long_sentence(e, g):
    return tf.shape(e)[0] <= 13

train_data = train_data.filter(remove_long_sentence)
valid_data = valid_data.filter(remove_long_sentence)

In [34]:
def pad_english(e, g):
    return tf.pad(e, paddings = [[13-tf.shape(e)[0],0], [0,0]], mode='CONSTANT', constant_values=0), g

train_data = train_data.map(pad_english)
valid_data = valid_data.map(pad_english)

In [35]:
train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [36]:
train_data.element_spec

(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))

In [37]:
valid_data.element_spec

(TensorSpec(shape=(None, None, 128), dtype=tf.float32, name=None),
 TensorSpec(shape=(None, 14), dtype=tf.int32, name=None))

In [41]:
for e, g in train_data.take(1):
    print(e.shape)

(16, 13, 128)


In [42]:
for e, g in valid_data.take(1):
    print(g)

tf.Tensor(
[[   1    5 1904 5289    3    2    0    0    0    0    0    0    0    0]
 [   1 3787    8    7    2    0    0    0    0    0    0    0    0    0]
 [   1 2607   28   29    3    2    0    0    0    0    0    0    0    0]
 [   1   64   31 1384    9    2    0    0    0    0    0    0    0    0]
 [   1    8   16   44 5052    3    2    0    0    0    0    0    0    0]
 [   1    8   35 1155    3    2    0    0    0    0    0    0    0    0]
 [   1   81   17   31   11 1213    7    2    0    0    0    0    0    0]
 [   1   86    8   53 1188   41    3    2    0    0    0    0    0    0]
 [   1   26   35   28  555    3    2    0    0    0    0    0    0    0]
 [   1   10    6 4993  279    3    2    0    0    0    0    0    0    0]
 [   1   38 2222   10   52    7    2    0    0    0    0    0    0    0]
 [   1    4   18  897  112   19  693    9    2    0    0    0    0    0]
 [   1    4   94 5658    3    2    0    0    0    0    0    0    0    0]
 [   1   38   16    5    7    2    0    

## 3. Create the custom layer
You will now create a custom layer to add the learned end token embedding to the encoder model:

![Encoder schematic](data/neural_translation_model_encoder.png)

You should now build the custom layer.
* Using layer subclassing, create a custom layer that takes a batch of English data examples from one of the Datasets, and adds a learned embedded ‘end’ token to the end of each sequence. 
* This layer should create a TensorFlow Variable (that will be learned during training) that is 128-dimensional (the size of the embedding space). _Hint: you may find it helpful in the call method to use the tf.tile function to replicate the end token embedding across every element in the batch._
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and print the shape. Test the custom layer by calling the layer on the English data batch Tensor and print the resulting Tensor shape (the layer should increase the sequence length by one).

In [46]:
from tensorflow.keras.models import  Sequential, Model
from tensorflow.keras.layers import Layer, Concatenate, Input, Masking, LSTM, Embedding, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

class CustomLayer(Layer):

    def __init__(self, **kwargs):
        super(CustomLayer, self).__init__(**kwargs)
        self.embed = tf.Variable(initial_value=tf.zeros(shape=(1,128), dtype='float32'))
        
    def call(self, inputs):
        x = tf.tile(self.embed, [tf.shape(inputs)[0], 1])
        x = tf.expand_dims(x, axis=1)
        return Concatenate(axis=1)([inputs, x])

In [47]:
custom_layer = CustomLayer()

In [50]:
e, g = next(iter(train_data.take(1)))
print(e.shape)
o = custom_layer(e)
o.shape

TensorShape([16, 14, 128])

## 4. Build the encoder network
The encoder network follows the schematic diagram above. You should now build the RNN encoder model.
* Using the functional API, build the encoder network according to the following spec:
    * The model will take a batch of sequences of embedded English words as input, as given by the Dataset objects.
    * The next layer in the encoder will be the custom layer you created previously, to add a learned end token embedding to the end of the English sequence.
    * This is followed by a Masking layer, with the `mask_value` set to the distinct padding value you used when you padded the English sequences with the Dataset preprocessing above.
    * The final layer is an LSTM layer with 512 units, which also returns the hidden and cell states.
    * The encoder is a multi-output model. There should be two output Tensors of this model: the hidden state and cell states of the LSTM layer. The output of the LSTM layer is unused.
* Using the Dataset `.take(1)` method, extract a batch of English data examples from the training Dataset and test the encoder model by calling it on the English data Tensor, and print the shape of the resulting Tensor outputs.
* Print the model summary for the encoder network.

In [64]:
inputs = Input(batch_shape = (None, 13, 128), name='input')
x = CustomLayer(name='custom_layer')(inputs)
x = Masking(mask_value=0, name='masking_layer')(x)
x, h, c = LSTM(units=512, return_state=True, name='lstm')(x)
encoder_model = Model(inputs = inputs, outputs = [h, c], name='encoder')

In [58]:
out = encoder_model(next(iter(train_data.take(1))))
print(out[0].shape, out[1].shape)
out

(16, 512) (16, 512)


[<tf.Tensor: id=8120, shape=(16, 512), dtype=float32, numpy=
 array([[ 0.0034057 ,  0.01008304,  0.00151506, ..., -0.0133437 ,
          0.00653079, -0.01176458],
        [ 0.0034057 ,  0.01008304,  0.00151506, ..., -0.0133437 ,
          0.00653079, -0.01176458],
        [ 0.00011291,  0.02568938,  0.00166299, ..., -0.00372866,
          0.01224773, -0.00191311],
        ...,
        [ 0.00319855,  0.01427315, -0.00189705, ..., -0.002008  ,
          0.02839963, -0.00543766],
        [-0.00552609,  0.01956979,  0.00704735, ...,  0.00197189,
          0.00803735, -0.00699146],
        [-0.00552609,  0.01956979,  0.00704735, ...,  0.00197189,
          0.00803735, -0.00699146]], dtype=float32)>,
 <tf.Tensor: id=8121, shape=(16, 512), dtype=float32, numpy=
 array([[ 0.00695205,  0.02061531,  0.00301814, ..., -0.02664329,
          0.01281457, -0.02357776],
        [ 0.00695205,  0.02061531,  0.00301814, ..., -0.02664329,
          0.01281457, -0.02357776],
        [ 0.00023065,  0.052539

In [65]:
encoder_model.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 13, 128)]         0         
_________________________________________________________________
custom_layer (CustomLayer)   (None, 14, 128)           128       
_________________________________________________________________
masking_layer (Masking)      (None, 14, 128)           0         
_________________________________________________________________
lstm (LSTM)                  [(None, 512), (None, 512) 1312768   
Total params: 1,312,896
Trainable params: 1,312,896
Non-trainable params: 0
_________________________________________________________________


## 5. Build the decoder network
The decoder network follows the schematic diagram below. 

![Decoder schematic](data/neural_translation_model_decoder.png)

You should now build the RNN decoder model.
* Using Model subclassing, build the decoder network according to the following spec:
    * The initializer should create the following layers:
        * An Embedding layer with vocabulary size set to the number of unique German tokens, embedding dimension 128, and set to mask zero values in the input.
        * An LSTM layer with 512 units, that returns its hidden and cell states, and also returns sequences.
        * A Dense layer with number of units equal to the number of unique German tokens, and no activation function.
    * The call method should include the usual `inputs` argument, as well as the additional keyword arguments `hidden_state` and `cell_state`. The default value for these keyword arguments should be `None`.
    * The call method should pass the inputs through the Embedding layer, and then through the LSTM layer. If the `hidden_state` and `cell_state` arguments are provided, these should be used for the initial state of the LSTM layer. _Hint: use the_ `initial_state` _keyword argument when calling the LSTM layer on its input._
    * The call method should pass the LSTM output sequence through the Dense layer, and return the resulting Tensor, along with the hidden and cell states of the LSTM layer.
* Using the Dataset `.take(1)` method, extract a batch of English and German data examples from the training Dataset. Test the decoder model by first calling the encoder model on the English data Tensor to get the hidden and cell states, and then call the decoder model on the German data Tensor and hidden and cell states, and print the shape of the resulting decoder Tensor outputs.
* Print the model summary for the decoder network.

In [89]:
class Decoder(Model):
    
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)
        self.embed = Embedding(input_dim=len(tokenizer.index_word)+1, output_dim=128, mask_zero=True, name='embedding_layer')
        self.lstm = LSTM(units = 512, return_state = True, return_sequences = True, name='lstm_layer')
        self.dense = Dense(len(tokenizer.index_word), name='dense_layer')
        
    def call(self, inputs, hidden_state = None, cell_state = None):
        x = self.embed(inputs)
        if hidden_state is not None and cell_state is not None:
            x, hidden_state, cell_state = self.lstm(x, initial_state = [hidden_state, cell_state])
        else:    
            x = self.lstm(x)
        x = self.dense(x)
        return x, hidden_state, cell_state

In [90]:
decoder_model = Decoder(name='decoder')
e, g_in = next(iter(train_data.take(1)))
h, c = encoder_model(e)
g_out, h, c = decoder_model(g_in, h, c)
print(g_out.shape, h.shape, c.shape)
g_out

(16, 14, 5743) (16, 512) (16, 512)


<tf.Tensor: id=22185, shape=(16, 14, 5743), dtype=float32, numpy=
array([[[ 3.1904574e-04, -2.1032616e-03, -2.7888743e-03, ...,
          2.0270820e-03, -7.2138384e-04,  1.2344717e-03],
        [-2.3266161e-04, -1.0980426e-03, -1.5519867e-03, ...,
          3.0956718e-03,  1.8840400e-04, -4.2528764e-04],
        [ 7.3767413e-04, -1.1167943e-04, -1.5705292e-03, ...,
          2.2907499e-03, -8.4343739e-04, -4.9658644e-04],
        ...,
        [ 4.3238490e-04,  5.6293682e-04,  4.5666401e-04, ...,
          9.3041896e-04, -1.8773051e-03, -1.8907028e-03],
        [ 4.3238490e-04,  5.6293682e-04,  4.5666401e-04, ...,
          9.3041896e-04, -1.8773051e-03, -1.8907028e-03],
        [ 4.3238490e-04,  5.6293682e-04,  4.5666401e-04, ...,
          9.3041896e-04, -1.8773051e-03, -1.8907028e-03]],

       [[ 3.1904574e-04, -2.1032616e-03, -2.7888743e-03, ...,
          2.0270820e-03, -7.2138384e-04,  1.2344717e-03],
        [ 1.3789127e-03, -2.5380531e-04, -2.7858184e-03, ...,
          9.74497

In [91]:
decoder_model.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  multiple                  735232    
_________________________________________________________________
lstm_layer (LSTM)            multiple                  1312768   
_________________________________________________________________
dense_layer (Dense)          multiple                  2946159   
Total params: 4,994,159
Trainable params: 4,994,159
Non-trainable params: 0
_________________________________________________________________


## 6. Make a custom training loop
You should now write a custom training loop to train your custom neural translation model.
* Define a function that takes a Tensor batch of German data (as extracted from the training Dataset), and returns a tuple containing German inputs and outputs for the decoder model (refer to schematic diagram above).
* Define a function that computes the forward and backward pass for your translation model. This function should take an English input, German input and German output as arguments, and should do the following:
    * Pass the English input into the encoder, to get the hidden and cell states of the encoder LSTM.
    * These hidden and cell states are then passed into the decoder, along with the German inputs, which returns a sequence of outputs (the hidden and cell state outputs of the decoder LSTM are unused in this function).
    * The loss should then be computed between the decoder outputs and the German output function argument.
    * The function returns the loss and gradients with respect to the encoder and decoder’s trainable variables.
    * Decorate the function with @tf.function
* Define and run a custom training loop for a number of epochs (for you to choose) that does the following:
    * Iterates through the training dataset, and creates decoder inputs and outputs from the German sequences.
    * Updates the parameters of the translation model using the gradients of the function above and an optimizer object.
    * Every epoch, compute the validation loss on a number of batches from the validation and save the epoch training and validation losses.
* Plot the learning curves for loss vs epoch for both training and validation sets.

_Hint: This model is computationally demanding to train. The quality of the model or length of training is not a factor in the grading rubric. However, to obtain a better model we recommend using the GPU accelerator hardware on Colab._

In [77]:
def get_german_decoder_data(german):
    return german[:, :-1], german[:,1:]    

In [78]:
e, g = next(iter(train_data.take(1)))
print(e.shape, g.shape)
g_in, g_out = get_german_decoder_data(g)
print(g_in.shape, g_out.shape)

(16, 13, 128) (16, 14)
(16, 13) (16, 13)


In [106]:
@tf.function
def forward_backward(encoder_model, decoder_model, e, g_in, g_out, loss):
    with tf.GradientTape() as tape:
        h, c = encoder_model(e)
        d_g_out, h, c = decoder_model(g_in, h, c)
        cur_loss = loss(g_out, d_g_out)
        grads = tape.gradient(cur_loss, encoder_model.trainable_variables + decoder_model.trainable_variables)
    return cur_loss, grads

In [107]:
def train_encoder_decoder(encoder_model, decoder_model, num_epochs, train_data, valid_data, valid_steps, 
                          optimizer, loss, grad_fn):
    train_losses = []
    val_loasses = []
    for epoch in range(num_epochs):
        train_epoch_loss_avg = tf.keras.metrics.Mean()
        val_epoch_loss_avg = tf.keras.metrics.Mean()
        for e, g in train_data:
            g_in, g_out = get_german_decoder_data(g)
            train_loss, grads = grad_fn(encoder_model, decoder_model, e, g_in, g_out, loss)
            optimizer.apply_gradients(zip(grads, encoder_model.trainable_variables + decoder_model.trainable_variables))
            train_epoch_loss_avg.update_state(train_loss)    
        for e_v, g_v in valid_data.take(valid_steps):
            g_v_in, g_v_out = get_german_decoder_data(g_v)
            d_g_v_out, _, _ = decoder_model(g_v_in)
            val_loss = loss(g_v_out, d_g_v_out)
            val_epoch_loss_avg.update_state(val_loss)            
        train_losses.append(train_epoch_loss_avg.result())
        val_loasses.append(val_epoch_loss_avg.result())
    return train_losses, val_loasses

In [108]:
optimizer_obj = Adam(learning_rate = 0.001)
loss_obj = SparseCategoricalCrossentropy()

In [ ]:
train_loss_results, valid_loss_results = train_encoder_decoder(encoder_model, decoder_model, 5, train_data, valid_data, 10,
                                                          optimizer_obj, loss_obj, forward_backward)

In [ ]:
fig, axes = plt.subplots(1, 2, sharex=True, figsize=(12, 5))

axes[0].set_xlabel("Epochs", fontsize=14)
axes[0].set_ylabel("Loss", fontsize=14)
axes[0].set_title('Train Loss vs epochs')
axes[0].plot(train_loss_results)

axes[1].set_title('Validation Loss vs epochs')
axes[1].set_ylabel("Loss", fontsize=14)
axes[1].set_xlabel("Epochs", fontsize=14)
axes[1].plot(valid_loss_results)
plt.show()

## 7. Use the model to translate
Now it's time to put your model into practice! You should run your translation for five randomly sampled English sentences from the dataset. For each sentence, the process is as follows:
* Preprocess and embed the English sentence according to the model requirements.
* Pass the embedded sentence through the encoder to get the encoder hidden and cell states.
* Starting with the special  `"<start>"` token, use this token and the final encoder hidden and cell states to get the one-step prediction from the decoder, as well as the decoder’s updated hidden and cell states.
* Create a loop to get the next step prediction and updated hidden and cell states from the decoder, using the most recent hidden and cell states. Terminate the loop when the `"<end>"` token is emitted, or when the sentence has reached a maximum length.
* Decode the output token sequence into German text and print the English text and the model's German translation.

In [ ]:
n = 5
indices = np.random.choice(len(english_sentences), 5)
for i in indices:
    e = embedding_layer(english_sentences[i])
    if e.shape[0] > 13:
        print('long sentence, skipping')
    else:
        e = tf.pad(e, paddings = [[13-tf.shape(e)[0],0], [0,0]], mode='CONSTANT', constant_values=0)
        h, c = encoder_model(e)
        g_t = []
        g_out, h, c = decoder_model(tokenize.word_index['<start>'], h, c)
        g_t.append('<start>)')
        while tokenizer.index_word(g_out) != '<end>': 
            g_out, h, c = decoder_model(g_in, h, c)
            g_in = g_out
            g_t.append(tokenizer.index_word.get(g_out, 'UNK'))
        print(f'English Text: {english_sentences[i]}')
        print(f'German Translation: {" ".join(g_t)}')
              print()